In [1]:
#import pandas as pd
import polars as pl
import numpy as np
import json
from datetime import datetime
import matplotlib.pyplot as plt
import os

fase = '01_LecturaDatos'

In [2]:
with open('gen_config.json', 'r') as file:
    gen_config =json.load(file)

In [3]:
folder = gen_config['folder']

#entradas

#salidas
path_group = gen_config['path_group']
path_prod_stats = gen_config['path_prod_stats']
path_prod_data = gen_config['path_prod_data']
path_stock_data = gen_config['path_stock_data']
path_overall_prod_stats = gen_config['path_overall_prod_stats']
#variables

print(f"{'COMIENZA':-^100}")
print(f"{fase:-^100}")

----------------------------------------------COMIENZA----------------------------------------------
------------------------------------------01_LecturaDatos-------------------------------------------


In [4]:
df = pl.read_csv('../sell-in.txt', separator='\t')
prod_a_predecir = pl.read_csv('../productos_a_predecir.txt', separator='\t')
prod_data = pl.read_csv('../tb_productos_descripcion.txt', separator='\t')
stock_data = pl.read_csv('../tb_stocks.txt', separator='\t')

Retenemos solo los productos en la lista de Productos A Predecir (los 780)

In [5]:
print(f"sell-in Shape: ({df.shape[0]:>9_d},{df.shape[1]:_d}, productos unicos: {df['product_id'].unique().count()})")
df = df.filter(pl.col('product_id').is_in(prod_a_predecir['product_id']))
print(f"df Shape     : ({df.shape[0]:>9_d},{df.shape[1]:_d}, productos unicos: {df['product_id'].unique().count()})")

sell-in Shape: (2_945_818,7, productos unicos: 1233)
df Shape     : (2_293_481,7, productos unicos: 780)


Creamos un dataframe con agregacion a nivel producto-cliente

en adelante usando "prodcust" para referinos a una combinacion de producto-cliente

In [6]:
df_grouped_prodcust = (
    df.group_by(['periodo', 'product_id', 'customer_id'])
    .agg(pl.col('tn').sum())
)
print(f"df_grouped_prodcust Shape: ({df_grouped_prodcust.shape[0]:>9_d},{df_grouped_prodcust.shape[1]:_d}, productos unicos: {df_grouped_prodcust['product_id'].unique().count()})")

df_grouped_prodcust Shape: (2_293_481,4, productos unicos: 780)


prod_stats va a retener datos estadisticos por prodcust

In [7]:
prod_stats = df_grouped_prodcust.group_by(['product_id','customer_id']).agg(
    [
        pl.min('periodo').alias('primer_periodo'),
        pl.max('periodo').alias('ultimo_periodo'),
        pl.len().alias('values'),
        pl.sum('tn').alias('total_tn'),
    ]
).sort(['product_id', 'customer_id'])

prod_stats = prod_stats.with_columns(
    (pl.col("primer_periodo").cast(pl.Utf8) + "01").str.to_date("%Y%m%d"),
    (pl.col("ultimo_periodo").cast(pl.Utf8) + "01").str.to_date("%Y%m%d")
)

print(f"prod_stats Shape:    ({prod_stats.shape[0]:>9_d},{prod_stats.shape[1]:_d}, productos unicos: {prod_stats['product_id'].unique().count()})")

prod_stats Shape:    (  262_805,6, productos unicos: 780)


ahora un dataframe similar pero con agrupacion a nivel producto

de particular utilidad para el primer y ultimo periodo de cada producto, que se usa para rellenar ceros

In [8]:
df_grouped_prod = (
    df.group_by(['periodo', 'product_id'])
    .agg(pl.col('tn').sum())
)

weight stats con la intencion de pasarlo al parametro weight del modelo en las ultrimas etapas

In [9]:
weight_stats = df_grouped_prod.filter(pl.col('periodo') >= 201901).group_by(['product_id']).agg(
    [
        pl.sum('tn').alias('tot_weight'),
        pl.mean('tn').alias('avg_weight'),
        pl.median('tn').alias('med_weight'),
    ]
).sort(['product_id'])

overall_prod_stats va a retener datos estadisticos por producto

In [10]:
overall_prod_stats = df_grouped_prod.group_by(['product_id']).agg(
    [
        pl.min('periodo').alias('primer_periodo'),
        pl.max('periodo').alias('ultimo_periodo'),
        pl.len().alias('values'),
        pl.sum('tn').alias('total_tn'),
        pl.min('tn').alias('min_tn'),
        pl.mean('tn').alias('average_tn'),
        pl.median('tn').alias('median_tn'),
        pl.std('tn',0).alias('std_dev_tn'),
        (pl.col('tn').quantile(0.75)).alias('Q3_tn'),
        (pl.col('tn').quantile(0.25)).alias('Q1_tn'),
        pl.max('tn').alias('max_tn'),
    ]
).sort(['product_id'])

overall_prod_stats = overall_prod_stats.with_columns(
    [
        (pl.col('Q3_tn') - pl.col('Q1_tn')).alias('iqr_tn'),
    ]
).sort(['product_id'])

overall_prod_stats = overall_prod_stats.with_columns(
    (pl.col("primer_periodo").cast(pl.Utf8) + "01").str.to_date("%Y%m%d"),
    (pl.col("ultimo_periodo").cast(pl.Utf8) + "01").str.to_date("%Y%m%d")
)

overall_prod_stats = overall_prod_stats.join(weight_stats, on='product_id', how='left', coalesce=True)

combinaciones: todos los prodcust existentes en el datafram

periodos: la lista completa de periodos

In [11]:
combinaciones = df_grouped_prodcust.select(["product_id", "customer_id"]).unique().sort(["product_id",'customer_id'])
todos_periodos = df_grouped_prodcust.select(["periodo"]).unique().sort("periodo")

tomamos COMBINACIONES y con un cross join con PERIODOS se obtiene el universo de datos posibles para todas las series

In [12]:
df_completo = combinaciones.join(todos_periodos, how="cross")

con este primer join se obtiene el primer y ultimo periodo a nivel producto

tambien agregamos "periodo_dt" en formato fecha

In [13]:
df_completo = df_completo.join(overall_prod_stats, on=["product_id"], how="left", coalesce=True).select(["product_id", "customer_id", "periodo", "primer_periodo", "ultimo_periodo"])
df_completo = df_completo.with_columns([
    ((pl.col("periodo").cast(pl.Utf8) + "01").str.to_date("%Y%m%d")).alias("periodo_dt"),  
])

este filtro controla q la linea valide q el dato de periodo esté dentro de la franja de existencia del producto

In [14]:
df_completo = df_completo.filter((pl.col('periodo_dt') >= pl.col('primer_periodo')) & (pl.col('periodo_dt') <= pl.col('ultimo_periodo')))

con esto se recupera el dato de tn para cada fila, como es un left join, los que tengan el dato, lo obtienen

los que queden nulos, son periodos donde el producto ya existe pero ese cliente no hizo una compra, entonces se imputan a cero

In [15]:
df_completo = df_completo.join(df_grouped_prodcust, on=['product_id', 'customer_id','periodo'], how='left', coalesce=True)
df_completo = df_completo.with_columns(
    pl.col("tn").fill_null(0)
)
df_completo = df_completo.drop(['primer_periodo','ultimo_periodo','periodo_dt'])

calculamos la feature "presentacion" en bas eal sku_size
- el maximo es grande
- el minimo es chico
- si maximo y minimo coinciden es unico
- caso contrario es medio

In [16]:
prod_data_skuinfo = prod_data.select(['cat1', 'cat2', 'cat3', 'brand','descripcion','sku_size'])
prod_data_skuinfo = prod_data_skuinfo.set_sorted("cat1", "cat2", "cat3", "brand", "descripcion")
prod_data_skuinfo = prod_data_skuinfo.group_by(['cat1', 'cat2', 'cat3', 'brand', 'descripcion']).agg([
    pl.col("sku_size").max().alias("max_skusize"),
    pl.col("sku_size").min().alias("min_skusize"),
    ])
prod_data = prod_data.join(prod_data_skuinfo, on=['cat1', 'cat2', 'cat3', 'brand', 'descripcion'], how='left', coalesce=True)


In [17]:
conditional_expr = (
    pl.when((pl.col('sku_size') == pl.col('max_skusize')) & (pl.col('sku_size') == pl.col('min_skusize')) )
    .then(pl.lit("Unico"))
    .when((pl.col('sku_size') == pl.col('max_skusize')))
    .then(pl.lit("Grande"))
    .when((pl.col('sku_size') == pl.col('min_skusize')))
    .then(pl.lit("Chico"))
    .otherwise(pl.lit("Medio"))
)

prod_data = prod_data.with_columns(conditional_expr.cast(pl.Categorical).alias('presentacion'))

In [18]:
#df_completo.filter((pl.col('product_id') == 20667) & (pl.col('customer_id') == 10427))

Reporte de datos a descartar

prodcust que no hayan tenido ventas desde junio y que no tengan 12 datos en total

se ve el porcentaje del total de toneladas q representan (2%)

In [19]:
rep_tot = prod_stats['total_tn'].sum()
rep_tot_tn = []
rep_count = []
rep_tot_pre = []
rep_count_pre = []


for i in range(12):
    rep_tot_tn.append(      prod_stats.filter((pl.col('values') <= i+1))['total_tn'].sum())
    rep_count.append(       prod_stats.filter((pl.col('values') <= i+1))['total_tn'].count())
    rep_tot_pre.append(     prod_stats.filter((pl.col('values') <= i+1) & (pl.col('ultimo_periodo') < pl.date(2019,6,1)))['total_tn'].sum())
    rep_count_pre.append(   prod_stats.filter((pl.col('values') <= i+1) & (pl.col('ultimo_periodo') < pl.date(2019,6,1)))['total_tn'].count())

rep = pl.DataFrame({'Datos presentes': range(1, 13), 'Total tn': rep_tot_tn, 'Cuenta ProdCust': rep_count, 'Total tn (pre junio)': rep_tot_pre, 'Cuenta ProdCust (pre junio)': rep_count_pre})
rep = rep.with_columns((pl.col('Total tn') / rep_tot).alias('Total tn %'))
rep = rep.with_columns((pl.col('Total tn (pre junio)') / rep_tot).alias('Total tn (pre junio) %'))

pl.Config.set_tbl_rows(20)
print("Reporte de datos a descartar")
print(rep)


Reporte de datos a descartar
shape: (12, 7)
┌───────────┬───────────────┬──────────┬───────────────┬───────────────┬────────────┬──────────────┐
│ Datos     ┆ Total tn      ┆ Cuenta   ┆ Total tn (pre ┆ Cuenta        ┆ Total tn % ┆ Total tn     │
│ presentes ┆ ---           ┆ ProdCust ┆ junio)        ┆ ProdCust (pre ┆ ---        ┆ (pre junio)  │
│ ---       ┆ f64           ┆ ---      ┆ ---           ┆ junio)        ┆ f64        ┆ %            │
│ i64       ┆               ┆ i64      ┆ f64           ┆ ---           ┆            ┆ ---          │
│           ┆               ┆          ┆               ┆ i64           ┆            ┆ f64          │
╞═══════════╪═══════════════╪══════════╪═══════════════╪═══════════════╪════════════╪══════════════╡
│ 1         ┆ 2526.49537    ┆ 44518    ┆ 1245.37808    ┆ 29204         ┆ 0.002251   ┆ 0.001109     │
│ 2         ┆ 6815.40544    ┆ 73667    ┆ 3031.92855    ┆ 45345         ┆ 0.006071   ┆ 0.002701     │
│ 3         ┆ 12256.17277   ┆ 96002    ┆ 4985.7

In [20]:
# calculo las filas a descartar

condition1 = pl.col('values') <= 12
condition2 = pl.col('ultimo_periodo') < pl.date(2019,6,1)

drop_prodcusts = prod_stats.filter((condition1 & condition2))[['product_id', 'customer_id']]

In [21]:
print(f"en df_completo se descartaron {drop_prodcusts.shape[0]:_d} / {prod_stats.shape[0]:_d} combinaciones de ProdCusts")
df_completo = df_completo.join(drop_prodcusts, on=['product_id', 'customer_id'], how='anti', coalesce=True)
print(f"Shape df_completo: {df_completo.shape[0]:_d}, {df_completo.shape[1]:_d}, productos unicos: {df_completo['product_id'].unique().shape[0]}")

print(f"en prod_stats se descartaron {drop_prodcusts.shape[0]:_d} / {prod_stats.shape[0]:_d} combinaciones de ProdCusts")
prod_stats = prod_stats.join(drop_prodcusts, on=['product_id', 'customer_id'], how='anti', coalesce=True)
print(f"Shape prod_stats: {prod_stats.shape[0]:_d}, {prod_stats.shape[1]:_d}, productos unicos: {prod_stats['product_id'].unique().shape[0]}")

en df_completo se descartaron 84_121 / 262_805 combinaciones de ProdCusts
Shape df_completo: 5_303_555, 4, productos unicos: 780
en prod_stats se descartaron 84_121 / 262_805 combinaciones de ProdCusts
Shape prod_stats: 178_684, 6, productos unicos: 780


In [22]:
temp = df_completo.group_by(['product_id','customer_id']).agg(
    [
        pl.sum('tn').alias('total_tn'),
        pl.min('tn').alias('min_tn'),
        pl.mean('tn').alias('average_tn'),
        pl.median('tn').alias('median_tn'),
        pl.std('tn',0).alias('std_dev_tn'),
        (pl.col('tn').quantile(0.75)).alias('Q3_tn'),
        (pl.col('tn').quantile(0.25)).alias('Q1_tn'),
        pl.max('tn').alias('max_tn')
    ]
)

temp = temp.with_columns(
    [
        (pl.col('Q3_tn') - pl.col('Q1_tn')).alias('iqr_tn'),
    ]
)

prod_stats = prod_stats.join(temp, on=['product_id', 'customer_id'], how='left', coalesce=True)

print(f"prod_stats Shape:    ({prod_stats.shape[0]:>9_d},{prod_stats.shape[1]:_d}, productos unicos: {prod_stats['product_id'].unique().count()})")

prod_stats Shape:    (  178_684,15, productos unicos: 780)


In [23]:
pl.Config.set_tbl_rows(36)
print("Ejemplo serie con ceros")
print(df_completo.filter((pl.col('product_id')==21276) & (pl.col('customer_id')==10462)))

Ejemplo serie con ceros
shape: (10, 4)
┌────────────┬─────────────┬─────────┬─────────┐
│ product_id ┆ customer_id ┆ periodo ┆ tn      │
│ ---        ┆ ---         ┆ ---     ┆ ---     │
│ i64        ┆ i64         ┆ i64     ┆ f64     │
╞════════════╪═════════════╪═════════╪═════════╡
│ 21276      ┆ 10462       ┆ 201903  ┆ 0.00075 │
│ 21276      ┆ 10462       ┆ 201904  ┆ 0.0     │
│ 21276      ┆ 10462       ┆ 201905  ┆ 0.00148 │
│ 21276      ┆ 10462       ┆ 201906  ┆ 0.0     │
│ 21276      ┆ 10462       ┆ 201907  ┆ 0.0     │
│ 21276      ┆ 10462       ┆ 201908  ┆ 0.0     │
│ 21276      ┆ 10462       ┆ 201909  ┆ 0.0     │
│ 21276      ┆ 10462       ┆ 201910  ┆ 0.00075 │
│ 21276      ┆ 10462       ┆ 201911  ┆ 0.0     │
│ 21276      ┆ 10462       ┆ 201912  ┆ 0.0     │
└────────────┴─────────────┴─────────┴─────────┘


In [24]:
df_completo.write_parquet(f'{folder}/{path_group}')
prod_stats.write_parquet(f'{folder}/{path_prod_stats}')
overall_prod_stats.write_parquet(f'{folder}/{path_overall_prod_stats}')
prod_data.write_parquet(f'{folder}/{path_prod_data}')
stock_data.write_parquet(f'{folder}/{path_stock_data}')

In [25]:
print(f"{fase:-^100}")
print(f"{'FINALIZA':-^100}")

------------------------------------------01_LecturaDatos-------------------------------------------
----------------------------------------------FINALIZA----------------------------------------------


In [26]:
# prod_stats = df_grouped_prodcust.group_by(['product_id','customer_id']).agg(
#     [
#         pl.min('periodo').alias('primer_periodo'),
#         pl.max('periodo').alias('ultimo_periodo'),
#         pl.len().alias('values'),
#         pl.sum('tn').alias('total_tn'),
#         pl.min('tn').alias('min_tn'),
#         pl.mean('tn').alias('average_tn'),
#         pl.median('tn').alias('median_tn'),
#         pl.std('tn',0).alias('std_dev_tn'),
#         (pl.col('tn').quantile(0.75)).alias('Q3_tn'),
#         (pl.col('tn').quantile(0.25)).alias('Q1_tn'),
#         pl.max('tn').alias('max_tn')
#     ]
# ).sort(['product_id', 'customer_id'])

# prod_stats = prod_stats.with_columns(
#     [
#         (pl.col('Q3_tn') - pl.col('Q1_tn')).alias('IQR'),
#     ]
# ).sort(['product_id', 'customer_id'])

# prod_stats = prod_stats.with_columns(
#     (pl.col("primer_periodo").cast(pl.Utf8) + "01").str.to_date("%Y%m%d"),
#     (pl.col("ultimo_periodo").cast(pl.Utf8) + "01").str.to_date("%Y%m%d")
# )

# print(f"prod_stats Shape:    ({prod_stats.shape[0]:>9_d},{prod_stats.shape[1]:_d}, productos unicos: {prod_stats['product_id'].unique().count()})")

In [27]:
# check = df_grouped_prodcust.filter((pl.col('product_id') == 20032) & (pl.col('customer_id') == 10344))
# check

In [28]:
# print(f"""
#     total = {check['tn'].sum()}
#     mean = {check['tn'].mean()}
#     median = {check['tn'].median()}
#     std = {check['tn'].std(ddof=0)}
#     Q3 = {check['tn'].quantile(0.75)}
#     Q1 = {check['tn'].quantile(0.25)}
#     """
#     )

In [29]:
#prod_stats.filter((pl.col('product_id') == 20032) & (pl.col('customer_id') == 10344))

In [30]:
# from sklearn.preprocessing import RobustScaler
# scaler = RobustScaler()

# check = check.with_columns(
#     pl.lit(scaler.fit_transform(check['tn'].to_numpy().reshape(-1, 1))).alias('tn_robust_sklearn')
#     )
# check



In [31]:
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()

# #serie_test = df.filter((pl.col('product_id')==20001) & (pl.col('customer_id')==10001))
# serie_test = df.filter((pl.col('product_id')==20003) & (pl.col('customer_id')==10420))
# serie_test = serie_test.join(prod_stats[['product_id', 'customer_id', 'average_tn', 'std_dev_tn']], on=['product_id', 'customer_id'], how='left', coalesce=True)
# serie_test = serie_test.with_columns([
#     pl.lit(scaler.fit_transform(serie_test['tn'].to_numpy().reshape(-1, 1))).alias('tn_standard_sklearn')
# ])

# serie_test = serie_test.with_columns([
#     ((pl.col('tn') - pl.col('average_tn')) / pl.col('std_dev_tn')).alias('tn_standardmanual'),
# ])

# print(scaler.var_**0.5)

# serie_test = serie_test.with_columns(
#     pl.col("tn_standardmanual")
#     .fill_nan(0)
#     .replace([float('inf'), float('-inf')], 0)
# )

# serie_test

In [32]:
# from sklearn.preprocessing import RobustScaler
# scaler = RobustScaler()

# #serie_test = df.filter((pl.col('product_id')==20001) & (pl.col('customer_id')==10001))
# serie_test = df.filter((pl.col('product_id')==20003) & (pl.col('customer_id')==10420))
# serie_test = serie_test.join(prod_stats[['product_id', 'customer_id', 'median_tn', 'iqr_tn']], on=['product_id', 'customer_id'], how='left', coalesce=True)
# serie_test = serie_test.with_columns([
#     pl.lit(scaler.fit_transform(serie_test['tn'].to_numpy().reshape(-1, 1))).alias('tn_robust_sklearn')
# ])

# serie_test = serie_test.with_columns([
#     ((pl.col('tn') - pl.col('median_tn')) / pl.col('iqr_tn')).alias('tn_robustmanual'),
# ])

# print(scaler.scale_)

# serie_test = serie_test.with_columns(
#     pl.col("tn_robustmanual")
#     .fill_nan(0)
#     .replace([float('inf'), float('-inf')], 0)
# )

# serie_test

In [33]:
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# serie_test = serie_test.with_columns([
#     pl.lit(scaler.fit_transform(serie_test['tn'].to_numpy().reshape(-1, 1))).alias('tn_standard')
# ])

# serie_test = serie_test.with_columns([
#     ((pl.col('tn') - pl.col('average_tn')) / pl.col('std_dev_tn')).alias('tn_standardmanual'),
# ])

# serie_test